In [ ]:
import numpy as np
from qiskit.circuit import Delay
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import SamplerV2
from qiskit_ibm_runtime import QiskitRuntimeService
import warnings
from qiskit.primitives import StatevectorSampler

import sys
import os

# Add "../../.." to the sys.path to use SteaneCodeCircuit class
sys.path.append(os.path.abspath("../../.."))
sys.path.append(os.path.abspath(".."))
from objects.steane_code_circuit import SteaneCodeCircuit
from functions.transpile import get_layout
from utils import get_transpile, TVD, load_experiment, run_experiment

In [ ]:
ibm_token = ""
service = QiskitRuntimeService(channel="ibm_quantum", token=ibm_token)

In [ ]:
# Select backend
backend_name = 'ibm_sherbrooke'
backend = service.backend(backend_name)
sampler = SamplerV2(backend)
shots = 12800

In [ ]:
# Get the dt of the selected backend in nano seconds
print(f" > dt in seconds: {backend.configuration().dt} s")
dt = backend.configuration().dt * 10**9
print(f" > dt in ns: {round(dt, 3)} ns")

In [ ]:
t_array = [t*9_000 for t in range(0, 100, 10)]
print(f" > Delays in dt: {t_array}")
print(f" > Delays un us: {[round(t*dt * 10**(-3) , 3) for t in t_array]}")

In [ ]:
# Load experiment of delay without initial layout:

state = '-'
basis = 'all'
error_correction = False

logical_operations = None
initial_layout = None
t_array_unit = 'dt' # Is dt by default

circuits, qc_circuits, initial_layout, estimated_durations = load_experiment(
    state, 
    basis, 
    error_correction, 
    backend, 
    transpile_iterations=10,
    t_array=t_array, 
    t_array_unit = t_array_unit,
    logical_operations = logical_operations, 
    initial_layout=initial_layout)

In [ ]:
qc_circuits[0].draw("mpl")

In [ ]:
# Define the valid states for the experiment
valid_states = ['0000000', '0001111', '0110011', '0111100', '1010101', '1011010', '1100110', '1101001',
                '1111111', '1110000', '1001100', '1000011', '0101010', '0100101', '0011001', '0010110']

expected_distribution = {}
for i in valid_states:
    expected_distribution[i] = 1/len(valid_states)

print(f" > Expected distribution if there is no errors:\n {expected_distribution}")

In [ ]:
experiment_name = 'delay_-_measureAll_noCorrection'
json_file = os.path.abspath("experiments.json")
encoder_type = 'universal'
t_array_unit = 'dt'
estimated_durations_units = 'us'

run_experiment(
    json_file, 
    experiment_name, 
    circuits, 
    shots, 
    sampler, 
    expected_distribution, 
    initial_layout, 
    estimated_durations, 
    estimated_durations_units,
    encoder_type,
    t_array = t_array,
    t_array_unit=t_array_unit)